In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from mhm_dataprocessing import group_weekly

#### Read files


In [2]:
#mHM output
mhm_fluxes = xr.open_dataset(r"W:\VUB\_main_research\mHM\mhm_outputs\mhm_RMI\mhm_mon_fluxes.nc")
mrm_fluxes = xr.open_dataset(r"W:\VUB\_main_research\mHM\mhm_outputs\mhm_RMI\mRM_Fluxes_States.nc")

#soil hydraulic properties
soil_hydr_grid = xr.open_dataset(r"W:\VUB\_main_research\mHM\mhm_outputs\mhm_RMI\mHM_restart_001.nc")

#Mask outside of BE domain
mask = mhm_fluxes['SM_L02'].isel(time=0)
#mask the data
mhm_mask = mask.where(mask > 0.7)*0 + 1
#Apply mask to mhm fluxes
mhm_fluxes = mhm_fluxes.where(mhm_mask == 1)

### Plant Available Water Anomalies

Functions to calculate standardized soil moisture index and soil moisture anomaly according to the European Drought Observatory

In [ ]:
# Extract the soil moisture content at L1 (all variables SWC_L0X). Use wildcards to select all horizons


# For each var, extract the soil moisture content, field capacity, and wilting point at L1
def calculate_paw_smi(mhm_fluxes, soil_hydr_grid, sm_var, nhorizon):
    """
    Calculate the standardised plant available water (PAW) according to the European Drought Observatory (EDO)
    
    Parameters
    ----------
    mhm_fluxes : xarray.Dataset
        The mHM dataset containing the soil moisture content at L1
    
    soil_hydr_grid : xarray.Dataset
        The mHM dataset containing the soil hydraulic properties: field capacity and wilting point

    sm_var : str
        The variable name of the soil moisture content at L1 (e.g. 'SWC_L01')
    nhorizon : int
        The horizon being considered (e.g. 0, 1, 2, ...)
        The horizon number must correpsond to the sm var being use.
        SWC_L01 corresponds to horizon 0, SWC_L02 corresponds to horizon 1, SWC_L0n corresponds to horizon n-1

    Returns
    -------
    smi_EDO : xarray.DataArray
        The standardised soil moisture index according to the European Drought Observatory
    
    """
    # Select the soil moisture content at L1
    sm_vars = [sm_var for sm_var in mhm_fluxes.variables if 'SWC_L0' in sm_var]
    
    swc = mhm_fluxes[[sm_var]] 
    if sm_var not in sm_vars:
        raise ValueError(f"Variable {sm_var} not found in the mHM dataset. Please select one of the following: {sm_vars}")
    # Group the data into weekly averages
    swc_weekly = group_weekly(swc)

    # Select soil field capacity and wilting point at L1
    soil_water_limits = soil_hydr_grid[['L1_soilMoistFC', 'L1_wiltingPoint']].isel(
        L1_LandCoverPeriods=0,  # Remains fixed since there is only one land cover period
        L1_SoilHorizons= nhorizon      # Can be changed to the number of horizons in the mHM model
    ).assign_coords(
        lat=(("lon", "lat"), soil_hydr_grid['L1_domain_lat'].values),
        lon=(("lon", "lat"), soil_hydr_grid['L1_domain_lon'].values)
    )

    #check if nhorizon is 1 less than the last digit of the var
    if nhorizon != int(sm_var[-1]) - 1:
        raise ValueError(f"Horizon {nhorizon} does not correspond to variable {sm_var}. Please select the correct horizon.")

    # Rename nrows1 and ncols1 to lat and lon
    soil_water_limits = soil_water_limits.rename({'ncols1': 'lon', 'nrows1': 'lat'})

    #Extract Field capacity and wilting point as arrays and assign the dims of swc
    fc_array = soil_water_limits['L1_soilMoistFC'].values
    wp_array = soil_water_limits['L1_wiltingPoint'].values

    # Create DataArray for field capacity
    fc_da = xr.DataArray(fc_array, dims=('lat', 'lon'), coords={'lat': swc['lat'], 'lon': swc['lon']})

    # Create DataArray for wilting point
    wp_da = xr.DataArray(wp_array, dims=('lat', 'lon'), coords={'lat': swc['lat'], 'lon': swc['lon']})

    # Calculate mean of FC and wilting point
    theta_50 = (fc_da + wp_da) / 2

    # Calculate plant available water (paw)
    #paw = soil_water_limits['L1_soilMoistFC'] - soil_water_limits['L1_wiltingPoint']

    #Calculate Weekly plant available water (paw) scaled to 0-1
    paw_scaled = (swc_weekly - wp_da) / (fc_da - wp_da)

     # Calculate the weekly EDO SMI (According to European Drought Observatory)
    smi_EDO = 1 - 1/(1 +(swc_weekly/theta_50)**6)

    # Return both smi_EDO and paw_scaled in a dictionary
    return {
        'smi_EDO': smi_EDO,
        'paw_scaled': paw_scaled
    }

    #Example usage:
    # to access each of the returned values, use the following syntax:
    #result = calculate_paw_smi(mhm_fluxes, soil_hydr_grid, 'SWC_L01', 0)
    #smi_EDO = result['smi_EDO']
    #paw_scaled = result['paw_scaled']


#==>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

def calculate_sma(smi_EDO, reference_period, target_date):
    """
    Calculate the Soil Moisture Anomaly (SMA) for a particular (week, year) pair
    based on the corresponding week in the reference period of choice.

    Parameters
    ----------
    smi_EDO : xarray.DataArray
        The standardised soil moisture index according to the European Drought Observatory
    reference_period : tuple
        A tuple of two datetime objects representing the start and end of the reference period
        e.g. ('1996-01-01', '2016-12-31'). Must include the brackets.
    target_date : str
        A string representing the target date in the format 'mm/dd/yyyy' e.g. '01/15/1997'

    Returns
    -------
    sma : xarray.DataArray
        The soil moisture anomaly for the target date based on the reference period

    """
    #Calculate the weekly Soil Moisture Anomaly (SMA)
    #Reference period 1996-2016
    ref_data = smi_EDO.sel(time=slice(reference_period[0], reference_period[1]))

    #Calculate the mean for each week of the reference period
    ref_mean_weekly = ref_data.groupby('week').mean(dim='time')

    #Calculate the standard deviation for each week of the reference period
    ref_std_weekly = ref_data.groupby('week').std(dim='time')
    
    #Calculate the SMA
    #select a particular week and year after the reference period
    
    target_datetime = pd.to_datetime(target_date)
    target_week = (target_datetime.dayofyear - 1) // 7
    target_year = target_datetime.year

    # Filter for year, dropping everything else
    yeardata = smi_EDO.sel(time=smi_EDO['time'].dt.year == target_year)

    # Now filter for the target week, dropping everything else
    weekdata = yeardata.where(yeardata['week'] == target_week, drop=True)
    
    #SMA = (SWC - mean) / std
    weekly_sma = (weekdata - ref_mean_weekly.sel(week=target_week)) / ref_std_weekly.sel(week=target_week)

    if target_week not in ref_mean_weekly['week']:
        raise ValueError(f"Target week {target_week} not found in the reference period.Please check if your data has daily timesteps.")

    return weekly_sma

In [175]:
sma= calculate_sma(smi_EDO, ['1996-01-01', '2016-12-31'], '12/01/2021')

In [ ]:
#Example usage:
result = calculate_paw_smi(mhm_fluxes, soil_hydr_grid, 'SWC_L02', 1)
smi_EDO = result['smi_EDO']
paw_scaled = result['paw_scaled']

sma= calculate_sma(smi_EDO, ('1996-01-01', '2016-12-31'), '12/01/2021')

#### Group to Weekly Averages

In [148]:
# Compute the custom week number
weeks = ((mrm_fluxes['time'].dt.dayofyear - 1) // 7)  # Subtract 1 to ensure 01/01 is within week 0
years = mrm_fluxes['time'].dt.year  # Extract years

# Initialize a list to store weekly data
weekly_means = []

for year in np.unique(mrm_fluxes['time'].dt.year)[0:2]:  # Iterate over first 2 years
    for i in np.unique(weeks.values):  # Iterate over unique weeks
            
            #============verification df ====================
            dfdate = pd.DataFrame({'time':pd.to_datetime(mrm_fluxes['time'].values)})
            dfdate['doy'] = dfdate['time'].dt.dayofyear

            dfdate['week'] = dfdate['time'].dt.dayofyear // 7
        
            #================================================

            # Select data for the year
            datayear = mrm_fluxes.sel(time=mrm_fluxes['time'].dt.year == year)

            # Select data for the given week for the selected year
            #For a given year, the last week containes one or two days.
            dataweek = datayear.sel(time=(datayear['time'].dt.dayofyear - 1)// 7 == i)

            if dataweek['time'].size > 0:  # Ensure there is valid data
                startdate = dataweek['time'][0].values
                enddate = dataweek['time'][-1].values  # Last day of the selected week

                # Select data for the given time range and compute weekly mean
                weekly_data = mrm_fluxes.sel(time=slice(startdate, enddate)).mean(dim='time')

                # Assign the weekly end date as the new time coordinate
                weekly_data = weekly_data.assign_coords(time=enddate, week=i)

                # Append the computed weekly mean to the list
                weekly_means.append(weekly_data)

    # Combine all weekly data into a single xarray dataset along the "time" dimension
    weekly_ds = xr.concat(weekly_means, dim="time")

#### Export FC and WP as netcdf

In [11]:
sm_var = 'SWC_L03'
swc = mhm_fluxes[['SWC_L03']]  
nhorizon = 2   

# Select soil field capacity and wilting point at L1
soil_water_limits = soil_hydr_grid[['L1_soilMoistFC', 'L1_wiltingPoint']].isel(
    L1_LandCoverPeriods=0,  # Remains fixed since there is only one land cover period
    L1_SoilHorizons= nhorizon     # Can be changed to the number of horizons in the mHM model
).assign_coords(
    lat=(("lon", "lat"), soil_hydr_grid['L1_domain_lat'].values),
    lon=(("lon", "lat"), soil_hydr_grid['L1_domain_lon'].values)
)

#check if nhorizon is 1 less than the last digit of the var
if nhorizon != int(sm_var[-1]) - 1:
    raise ValueError(f"Horizon {nhorizon} does not correspond to variable {sm_var}. Please select the correct horizon.")

# Rename nrows1 and ncols1 to lat and lon
soil_water_limits = soil_water_limits.rename({'ncols1': 'lon', 'nrows1': 'lat'})

#Extract Field capacity and wilting point as arrays and assign the dims of swc
fc_array = soil_water_limits['L1_soilMoistFC'].values
wp_array = soil_water_limits['L1_wiltingPoint'].values

# Create DataArray for field capacity
fc_da = xr.DataArray(fc_array, dims=('lat', 'lon'), coords={'lat': swc['lat'], 'lon': swc['lon']})

#fc_da to dataset
fc_ds = fc_da.to_dataset(name='fc')

# Create DataArray for wilting point
wp_da = xr.DataArray(wp_array, dims=('lat', 'lon'), coords={'lat': swc['lat'], 'lon': swc['lon']})

#wp_da to dataset
wp_ds = wp_da.to_dataset(name='wp')


In [16]:
#export to fc and wp to netcdf
fc_ds.to_netcdf(r"E:\mHM__presentation\fc_L03.nc")
wp_ds.to_netcdf(r"E:\mHM__presentation\wp_L03.nc")